In [13]:
import argparse
import numpy as np
import pandas as pd
import yaml
from aqad.core import AQADFramework
from aqad.utils import load_config, get_model, get_quantization_scheme
from aqad.thresholding import AdaptiveThreshold
from data.data_loader import load_and_preprocess, DataLoader
from attacks.fgsm import fgsm_attack


config_path = "D:/mine/aqad-framework/config/generated_config.yaml"
data_path = "D:/mine/aqad-framework/synthetic_dataset.csv"

# Load configuration
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# Load and preprocess data
X, y, new_names = load_and_preprocess(config, data_path, one_hot_encode=False)

# Split data
loader = DataLoader(config)
X_train, X_test, y_train, y_test = loader.split_data(X, y)

print(X_train.shape, y_train.shape)

#converting it to a dataframe
temp = pd.DataFrame(X, columns=new_names)

All features: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19']
Categorical features: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4']
Numerical features: ['feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19']
Target variable: target
Preprocessed feature names: ['feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4']
One-hot encoding: No
(800, 20) (800,)


In [14]:

# Initialize base model
base_model = get_model(config['base_model'])

# Initialize quantization schemes
quantization_schemes = [get_quantization_scheme(scheme) for scheme in config['quantization_schemes']]

# Initialize AED models
aed_models = [get_model(model_config) for model_config in config['aed_models']]

# Initialize threshold model
threshold_model = AdaptiveThreshold(get_model(config['threshold_model']))

print("Base Model: ", base_model, "Quantization Scheme: ", quantization_schemes, "AED Model: ", aed_models, "Threshold model: ", threshold_model)

# Create AQAD framework
aqad = AQADFramework(base_model, quantization_schemes, aed_models, threshold_model, 
                     categorical_columns=['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4'],
                     feature_names=list(temp.columns))

# Train base model
base_model.fit(X_train, y_train)

# Generate adversarial examples
X_train_adv = fgsm_attack(base_model, pd.DataFrame(X_train, columns=list(temp.columns)), y_train, epsilon=0.1, 
                          categorical_columns=['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4'])

Base Model:  RandomForestClassifier(max_depth=10) Quantization Scheme:  [<function get_quantization_scheme.<locals>.<lambda> at 0x00000216EEEEF7E0>, <function get_quantization_scheme.<locals>.<lambda> at 0x00000216EEEEF420>] AED Model:  [RandomForestClassifier(max_depth=5, n_estimators=50), GradientBoostingClassifier(n_estimators=50)] Threshold model:  AdaptiveThreshold(base_estimator=LogisticRegression())


TypeError: AQADFramework.__init__() got an unexpected keyword argument 'feature_names'

In [12]:
# Initialize base model
base_model = get_model(config['base_model'])

# Initialize quantization schemes
quantization_schemes = [get_quantization_scheme(scheme) for scheme in config['quantization_schemes']]

# Initialize AED models
aed_models = [get_model(model_config) for model_config in config['aed_models']]

# Initialize threshold model
threshold_model = AdaptiveThreshold(get_model(config['threshold_model']))

print("Base Model: ", base_model, "Quantization Scheme: ", quantization_schemes, "AED Model: ", aed_models, "Threshold model: ", threshold_model)

# Create AQAD framework
aqad = AQADFramework(base_model, quantization_schemes, aed_models, threshold_model, 
                     categorical_columns=['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4'],
                     feature_names=list(temp.columns))

# Train base model
base_model.fit(X_train, y_train)

# Generate adversarial examples
X_train_adv = fgsm_attack(base_model, pd.DataFrame(X_train, columns=list(temp.columns)), y_train, epsilon=0.1, 
                          categorical_columns=['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4'])


Base Model:  RandomForestClassifier(max_depth=10) Quantization Scheme:  [<function get_quantization_scheme.<locals>.<lambda> at 0x00000216EEEEE8E0>, <function get_quantization_scheme.<locals>.<lambda> at 0x00000216EEEEF560>] AED Model:  [RandomForestClassifier(max_depth=5, n_estimators=50), GradientBoostingClassifier(n_estimators=50)] Threshold model:  AdaptiveThreshold(base_estimator=LogisticRegression())


TypeError: AQADFramework.__init__() got an unexpected keyword argument 'feature_names'

In [ ]:
# Combine original and adversarial examples
X_train_combined = np.vstack([X_train, X_train_adv])
y_train_combined = np.hstack([y_train, y_train])

# Train AQAD framework
aqad.fit(X_train_combined, y_train_combined)

# Evaluate
y_pred = aqad.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print(f"Test accuracy: {accuracy}")
